In [80]:
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')


filename_input = 'gap=0.2_result.csv'
filename_output = 'output_segment.json'

#filename_input = 'gap=1_result.csv'
#filename_output = 'output_no_segment.json'


# CSV 파일을 읽어옵니다.
csv_file = filename_input
df = pd.read_csv(csv_file)

# JSON 구조를 생성합니다.
output = []

# 고유한 filename을 찾습니다.
filenames = df['filename'].unique()


In [81]:
inner_list = [0] * 5  # 예시로 [0, 0, 0]으로 초기화
outer_list = [inner_list[:] for _ in range(20)]




for filename in filenames:
    print("processing", filename)
    file_data = {"performance_id": filename, "features": []}
    
    # 해당 filename에 대한 데이터를 필터링합니다.
    file_df = df[df['filename'] == filename]
    
    # 고유한 feature를 찾습니다.
    features = file_df['feature'].unique()

    for feature in features:

        feature_data = {"feature_type": int(feature), "intervals": []}
        
        # 해당 feature에 대한 데이터를 필터링합니다.
        feature_df = file_df[file_df['feature'] == feature]
        
        
        full = [False] * 20
        value = 0
        gap = 0
        count = 0
        start, end = -1, -1
        
        
        #print("feature_df")
        #print(feature_df)
        
        # 빈 compare_gap 데이터프레임 생성
        compare_gap = pd.DataFrame(columns=feature_df.columns)

        # for문을 돌며 row를 새로운 dataframe 'compare_gap'에 추가
        for _, row in feature_df.iterrows():
            compare_gap = compare_gap.append(row)
        
        #print("compare_gap")
        #print(compare_gap)

        # 'compare_gap'을 'gap' column의 값이 큰 순으로 정렬
        compare_gap = compare_gap.sort_values(by='gap', ascending=False)
        
        #print("compare_gap ordered")
        #print(compare_gap)

        # 'compare_gap'의 첫번째 행부터 끝 행까지 아래 for문 동작
        for _, row in compare_gap.iterrows():
            start_idx = int(row['start'])
            end_idx = int(row['end'])

            # full[start_idx:end_idx]가 모두 False면:
            #print("full check")
            #print(full)
            if all(not full[i] for i in range(start_idx, end_idx+1)):
                # full[start_idx:end_idx]를 True로 만들기
                for i in range(start_idx, end_idx+1):
                    full[i] = True
                    #print("full update")
                    #print(full)

                # 이전 interval 값 처리
                if start > 0:
                    interval = {
                        "interval": [start, end],
                        "value": round(value / count,4)
                    }
                    value_id = round(value / count,4)
                    outer_list[feature][4] += int(row['end']) -int(row['start'])
                    outer_list[feature][3] += 1 #count
                    if value_id > 4:
                        outer_list[feature][0] += 1
                    elif value_id < 4:
                        outer_list[feature][2] += 1
                    else:
                        outer_list[feature][1] += 1
                        
                    feature_data["intervals"].append(interval)
                    #print("interval1")
                    #print(interval)  # interval을 저장하거나 처리하는 부분

                # 현재 row 값 업데이트
                value = float(row['value'])
                gap = float(row['gap'])
                start = start_idx
                end = end_idx
                count = 1
            else:
                # full[start_idx:end_idx] 중 하나라도 True면, 즉 겹치면.
                
                # gap 같은 같은 값이면 value 평균 내기
                if gap == float(row['gap']):
                    value += float(row['value'])
                    count += 1
                # 다음 gqp row이면 이전 것 append 하고 변수 초기화
                elif gap > float(row['gap']) and count > 0:
                    interval = {
                        "interval": [start, end],
                        "value": round(value / count,4)
                    }
                    value_id = round(value / count,4)
                    outer_list[feature][4] += int(row['end']) -int(row['start'])
                    outer_list[feature][3] += 1 #count
                    if value_id > 4:
                        outer_list[feature][0] += 1
                    elif value_id < 4:
                        outer_list[feature][2] += 1
                    else:
                        outer_list[feature][1] += 1
                    
                    feature_data["intervals"].append(interval)
                    #print("interval2")
                    #print(interval)  # interval을 저장하거나 처리하는 부분
                    value = 0
                    count = 0
                    start = -1
                    end = -1
                else:
                    value = 0
                    count = 0
                    start = -1
                    end = -1
                

        # 마지막 interval 값 처리
        if start > 0:
            interval = {
                "interval": [start, end],
                "value": round(value / count,4)
            }
            value_id = round(value / count,4)
            outer_list[feature][4] += int(row['end']) -int(row['start'])
            outer_list[feature][3] += 1 #count
            if value_id > 4:
                outer_list[feature][0] += 1
            elif value_id < 4:
                outer_list[feature][2] += 1
            else:
                outer_list[feature][1] += 1
            feature_data["intervals"].append(interval)
            #print("interval3")
            #print(interval)  # interval을 저장하거나 처리하는 부분
            
        #print("last")
        #print(feature_data["intervals"])
        
        file_data["features"].append(feature_data)
    output.append(file_data)

# JSON 객체를 생성합니다.
json_data = {"output": output}

# JSON 파일로 저장합니다.
with open(filename_output, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

# 최종 결과만 출력
print("JSON 파일이 성공적으로 생성되었습니다.")

processing Beethoven_WoO80_var10_8bars_11_8
processing Beethoven_WoO80_var10_8bars_14_8
processing Beethoven_WoO80_var10_8bars_18_8
processing Beethoven_WoO80_var10_8bars_19_8
processing Beethoven_WoO80_var10_8bars_1_8
processing Beethoven_WoO80_var10_8bars_22_8
processing Beethoven_WoO80_var10_8bars_24_8
processing Beethoven_WoO80_var10_8bars_26_8
processing Beethoven_WoO80_var10_8bars_2_8
processing Beethoven_WoO80_var10_8bars_3_8
processing Beethoven_WoO80_var12_8bars_11_9
processing Beethoven_WoO80_var12_8bars_14_9
processing Beethoven_WoO80_var12_8bars_18_9
processing Beethoven_WoO80_var12_8bars_19_9
processing Beethoven_WoO80_var12_8bars_1_9
processing Beethoven_WoO80_var12_8bars_22_9
processing Beethoven_WoO80_var12_8bars_24_9
processing Beethoven_WoO80_var12_8bars_26_9
processing Beethoven_WoO80_var12_8bars_2_9
processing Beethoven_WoO80_var12_8bars_3_9
processing Beethoven_WoO80_var12_8bars_4_9
processing Beethoven_WoO80_var12_8bars_7_9
processing Beethoven_WoO80_var12_8bars_S

processing Schubert_D935_no.3_4bars_6_37
processing Schubert_D935_no.3_4bars_6_42
processing Schubert_D935_no.3_4bars_6_46
processing Schubert_D935_no.3_4bars_6_5
processing Schubert_D935_no.3_4bars_6_9
processing Schubert_D935_no.3_4bars_9_1
processing Schubert_D935_no.3_4bars_9_13
processing Schubert_D935_no.3_4bars_9_17
processing Schubert_D935_no.3_4bars_9_21
processing Schubert_D935_no.3_4bars_9_25
processing Schubert_D935_no.3_4bars_9_42
processing Schubert_D935_no.3_4bars_9_5
processing Schubert_D935_no.3_4bars_Score2_1
processing Schubert_D935_no.3_4bars_Score2_13
processing Schubert_D935_no.3_4bars_Score2_17
processing Schubert_D935_no.3_4bars_Score2_21
processing Schubert_D935_no.3_4bars_Score2_25
processing Schubert_D935_no.3_4bars_Score2_29
processing Schubert_D935_no.3_4bars_Score2_33
processing Schubert_D935_no.3_4bars_Score2_42
processing Schubert_D935_no.3_4bars_Score2_46
processing Schubert_D935_no.3_4bars_Score2_5
processing Schubert_D935_no.3_4bars_Score2_9
processin

processing Schubert_D960_mv2_8bars_5_09
processing Schubert_D960_mv2_8bars_5_11
processing Schubert_D960_mv2_8bars_5_13
processing Schubert_D960_mv2_8bars_5_14
processing Schubert_D960_mv2_8bars_5_15
processing Schubert_D960_mv2_8bars_5_17
processing Schubert_D960_mv2_8bars_6_01
processing Schubert_D960_mv2_8bars_6_02
processing Schubert_D960_mv2_8bars_6_03
processing Schubert_D960_mv2_8bars_6_04
processing Schubert_D960_mv2_8bars_6_06
processing Schubert_D960_mv2_8bars_6_07
processing Schubert_D960_mv2_8bars_6_08
processing Schubert_D960_mv2_8bars_6_09
processing Schubert_D960_mv2_8bars_6_10
processing Schubert_D960_mv2_8bars_6_13
processing Schubert_D960_mv2_8bars_6_14
processing Schubert_D960_mv2_8bars_6_15
processing Schubert_D960_mv2_8bars_6_17
processing Schubert_D960_mv2_8bars_7_01
processing Schubert_D960_mv2_8bars_7_02
processing Schubert_D960_mv2_8bars_7_03
processing Schubert_D960_mv2_8bars_7_04
processing Schubert_D960_mv2_8bars_7_05
processing Schubert_D960_mv2_8bars_7_07


processing Schubert_D960_mv3_8bars_12_06
processing Schubert_D960_mv3_8bars_12_07
processing Schubert_D960_mv3_8bars_12_08
processing Schubert_D960_mv3_8bars_12_09
processing Schubert_D960_mv3_8bars_12_10
processing Schubert_D960_mv3_8bars_12_11
processing Schubert_D960_mv3_8bars_12_12
processing Schubert_D960_mv3_8bars_12_13
processing Schubert_D960_mv3_8bars_12_14
processing Schubert_D960_mv3_8bars_12_15
processing Schubert_D960_mv3_8bars_12_16
processing Schubert_D960_mv3_8bars_12_17
processing Schubert_D960_mv3_8bars_12_18
processing Schubert_D960_mv3_8bars_12_19
processing Schubert_D960_mv3_8bars_12_20
processing Schubert_D960_mv3_8bars_12_21
processing Schubert_D960_mv3_8bars_12_22
processing Schubert_D960_mv3_8bars_12_23
processing Schubert_D960_mv3_8bars_12_24
processing Schubert_D960_mv3_8bars_12_25
processing Schubert_D960_mv3_8bars_12_26
processing Schubert_D960_mv3_8bars_12_27
processing Schubert_D960_mv3_8bars_12_29
processing Schubert_D960_mv3_8bars_12_31
processing Schub

processing Schubert_D960_mv3_8bars_6_27
processing Schubert_D960_mv3_8bars_6_28
processing Schubert_D960_mv3_8bars_6_30
processing Schubert_D960_mv3_8bars_6_31
processing Schubert_D960_mv3_8bars_6_33
processing Schubert_D960_mv3_8bars_6_34
processing Schubert_D960_mv3_8bars_6_35
processing Schubert_D960_mv3_8bars_6_36
processing Schubert_D960_mv3_8bars_6_37
processing Schubert_D960_mv3_8bars_7_02
processing Schubert_D960_mv3_8bars_7_03
processing Schubert_D960_mv3_8bars_7_04
processing Schubert_D960_mv3_8bars_7_06
processing Schubert_D960_mv3_8bars_7_08
processing Schubert_D960_mv3_8bars_7_10
processing Schubert_D960_mv3_8bars_7_14
processing Schubert_D960_mv3_8bars_7_15
processing Schubert_D960_mv3_8bars_7_16
processing Schubert_D960_mv3_8bars_7_17
processing Schubert_D960_mv3_8bars_7_18
processing Schubert_D960_mv3_8bars_7_19
processing Schubert_D960_mv3_8bars_7_20
processing Schubert_D960_mv3_8bars_7_21
processing Schubert_D960_mv3_8bars_7_22
processing Schubert_D960_mv3_8bars_7_23


In [82]:
outer_list

[[0, 0, 0, 0, 0],
 [380, 80, 1159, 1619, 1432],
 [839, 82, 911, 1832, 1534],
 [582, 65, 812, 1459, 1288],
 [1942, 74, 1223, 3239, 2896],
 [1166, 87, 1733, 2986, 2663],
 [416, 93, 795, 1304, 1066],
 [1071, 97, 670, 1838, 1539],
 [141, 38, 753, 932, 804],
 [898, 92, 704, 1694, 1491],
 [593, 100, 1152, 1845, 1595],
 [138, 28, 257, 423, 349],
 [329, 35, 743, 1107, 921],
 [437, 43, 656, 1136, 963],
 [489, 47, 282, 818, 683],
 [631, 68, 492, 1191, 928],
 [191, 43, 865, 1099, 954],
 [330, 37, 207, 574, 472],
 [547, 62, 826, 1435, 1225],
 [587, 44, 541, 1172, 1005]]

In [83]:

filename_input = 'gap=1_result.csv'
filename_output = 'output_no_segment.json'


# CSV 파일을 읽어옵니다.
csv_file = filename_input
df = pd.read_csv(csv_file)

# JSON 구조를 생성합니다.
output = []

# 고유한 filename을 찾습니다.
filenames = df['filename'].unique()


In [84]:
inner_list = [0] * 5  # 예시로 [0, 0, 0]으로 초기화
outer_list = [inner_list[:] for _ in range(20)]

for filename in filenames:
    print("processing", filename)
    file_data = {"performance_id": filename, "features": []}
    
    # 해당 filename에 대한 데이터를 필터링합니다.
    file_df = df[df['filename'] == filename]
    
    # 고유한 feature를 찾습니다.
    features = file_df['feature'].unique()

    for feature in features:

        feature_data = {"feature_type": int(feature), "intervals": []}
        
        # 해당 feature에 대한 데이터를 필터링합니다.
        feature_df = file_df[file_df['feature'] == feature]
        
        value = 0
        count = 0

        # 같은 feature에 대해서 value 합
        for _, row in feature_df.iterrows():
            start = int(row['start'])
            end = int(row['end'])
            
            value = float(row['value'])

            count += 1
                

        # 마지막 interval 값 처리
        if count > 0:
            interval = {
                "interval": [start, end],
                "value": round(value / count,4)
            }
            value_id = round(value / count,4)
            outer_list[feature][4] += int(row['end']) -int(row['start'])
            outer_list[feature][3] += 1 #count
            if value_id > 4:
                outer_list[feature][0] += 1
            elif value_id < 4:
                outer_list[feature][2] += 1
            else:
                outer_list[feature][1] += 1
            feature_data["intervals"].append(interval)
            #print("interval3")
            #print(interval)  # interval을 저장하거나 처리하는 부분
            
        #print("last")
        #print(feature_data["intervals"])
        
        file_data["features"].append(feature_data)
    output.append(file_data)

# JSON 객체를 생성합니다.
json_data = {"output": output}

# JSON 파일로 저장합니다.
with open(filename_output, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

# 최종 결과만 출력
print("JSON 파일이 성공적으로 생성되었습니다.")

processing Beethoven_WoO80_var10_8bars_11_8
processing Beethoven_WoO80_var10_8bars_14_8
processing Beethoven_WoO80_var10_8bars_18_8
processing Beethoven_WoO80_var10_8bars_19_8
processing Beethoven_WoO80_var10_8bars_1_8
processing Beethoven_WoO80_var10_8bars_22_8
processing Beethoven_WoO80_var10_8bars_24_8
processing Beethoven_WoO80_var10_8bars_26_8
processing Beethoven_WoO80_var10_8bars_2_8
processing Beethoven_WoO80_var10_8bars_3_8
processing Beethoven_WoO80_var12_8bars_11_9
processing Beethoven_WoO80_var12_8bars_14_9
processing Beethoven_WoO80_var12_8bars_18_9
processing Beethoven_WoO80_var12_8bars_19_9
processing Beethoven_WoO80_var12_8bars_1_9
processing Beethoven_WoO80_var12_8bars_22_9
processing Beethoven_WoO80_var12_8bars_24_9
processing Beethoven_WoO80_var12_8bars_26_9
processing Beethoven_WoO80_var12_8bars_2_9
processing Beethoven_WoO80_var12_8bars_3_9
processing Beethoven_WoO80_var12_8bars_4_9
processing Beethoven_WoO80_var12_8bars_7_9
processing Beethoven_WoO80_var12_8bars_S

processing Schubert_D935_no.3_4bars_9_25
processing Schubert_D935_no.3_4bars_9_42
processing Schubert_D935_no.3_4bars_9_5
processing Schubert_D935_no.3_4bars_Score2_1
processing Schubert_D935_no.3_4bars_Score2_13
processing Schubert_D935_no.3_4bars_Score2_17
processing Schubert_D935_no.3_4bars_Score2_21
processing Schubert_D935_no.3_4bars_Score2_25
processing Schubert_D935_no.3_4bars_Score2_29
processing Schubert_D935_no.3_4bars_Score2_33
processing Schubert_D935_no.3_4bars_Score2_42
processing Schubert_D935_no.3_4bars_Score2_46
processing Schubert_D935_no.3_4bars_Score2_5
processing Schubert_D935_no.3_4bars_Score2_9
processing Schubert_D935_no.3_4bars_Score_13
processing Schubert_D935_no.3_4bars_Score_17
processing Schubert_D935_no.3_4bars_Score_21
processing Schubert_D935_no.3_4bars_Score_25
processing Schubert_D935_no.3_4bars_Score_29
processing Schubert_D935_no.3_4bars_Score_33
processing Schubert_D935_no.3_4bars_Score_37
processing Schubert_D935_no.3_4bars_Score_42
processing Schu

processing Schubert_D960_mv2_8bars_6_09
processing Schubert_D960_mv2_8bars_6_10
processing Schubert_D960_mv2_8bars_6_13
processing Schubert_D960_mv2_8bars_6_14
processing Schubert_D960_mv2_8bars_6_15
processing Schubert_D960_mv2_8bars_6_17
processing Schubert_D960_mv2_8bars_7_01
processing Schubert_D960_mv2_8bars_7_02
processing Schubert_D960_mv2_8bars_7_03
processing Schubert_D960_mv2_8bars_7_04
processing Schubert_D960_mv2_8bars_7_05
processing Schubert_D960_mv2_8bars_7_07
processing Schubert_D960_mv2_8bars_7_08
processing Schubert_D960_mv2_8bars_7_09
processing Schubert_D960_mv2_8bars_7_10
processing Schubert_D960_mv2_8bars_7_11
processing Schubert_D960_mv2_8bars_7_12
processing Schubert_D960_mv2_8bars_7_13
processing Schubert_D960_mv2_8bars_7_14
processing Schubert_D960_mv2_8bars_7_15
processing Schubert_D960_mv2_8bars_7_16
processing Schubert_D960_mv2_8bars_7_17
processing Schubert_D960_mv2_8bars_8_01
processing Schubert_D960_mv2_8bars_8_02
processing Schubert_D960_mv2_8bars_8_04


processing Schubert_D960_mv3_8bars_12_33
processing Schubert_D960_mv3_8bars_12_34
processing Schubert_D960_mv3_8bars_12_35
processing Schubert_D960_mv3_8bars_12_36
processing Schubert_D960_mv3_8bars_1_01
processing Schubert_D960_mv3_8bars_1_02
processing Schubert_D960_mv3_8bars_1_03
processing Schubert_D960_mv3_8bars_1_04
processing Schubert_D960_mv3_8bars_1_05
processing Schubert_D960_mv3_8bars_1_06
processing Schubert_D960_mv3_8bars_1_07
processing Schubert_D960_mv3_8bars_1_08
processing Schubert_D960_mv3_8bars_1_09
processing Schubert_D960_mv3_8bars_1_10
processing Schubert_D960_mv3_8bars_1_11
processing Schubert_D960_mv3_8bars_1_12
processing Schubert_D960_mv3_8bars_1_13
processing Schubert_D960_mv3_8bars_1_14
processing Schubert_D960_mv3_8bars_1_15
processing Schubert_D960_mv3_8bars_1_16
processing Schubert_D960_mv3_8bars_1_17
processing Schubert_D960_mv3_8bars_1_18
processing Schubert_D960_mv3_8bars_1_19
processing Schubert_D960_mv3_8bars_1_20
processing Schubert_D960_mv3_8bars_1

processing Schubert_D960_mv3_8bars_7_24
processing Schubert_D960_mv3_8bars_7_25
processing Schubert_D960_mv3_8bars_7_26
processing Schubert_D960_mv3_8bars_7_27
processing Schubert_D960_mv3_8bars_7_29
processing Schubert_D960_mv3_8bars_7_30
processing Schubert_D960_mv3_8bars_7_31
processing Schubert_D960_mv3_8bars_7_32
processing Schubert_D960_mv3_8bars_7_33
processing Schubert_D960_mv3_8bars_7_35
processing Schubert_D960_mv3_8bars_7_36
processing Schubert_D960_mv3_8bars_7_37
processing Schubert_D960_mv3_8bars_8_01
processing Schubert_D960_mv3_8bars_8_02
processing Schubert_D960_mv3_8bars_8_04
processing Schubert_D960_mv3_8bars_8_05
processing Schubert_D960_mv3_8bars_8_06
processing Schubert_D960_mv3_8bars_8_09
processing Schubert_D960_mv3_8bars_8_10
processing Schubert_D960_mv3_8bars_8_12
processing Schubert_D960_mv3_8bars_8_13
processing Schubert_D960_mv3_8bars_8_14
processing Schubert_D960_mv3_8bars_8_15
processing Schubert_D960_mv3_8bars_8_16
processing Schubert_D960_mv3_8bars_8_17


In [85]:
outer_list

[[0, 0, 0, 0, 0],
 [26, 0, 828, 854, 6690],
 [34, 0, 844, 878, 6950],
 [29, 0, 812, 841, 6683],
 [45, 0, 842, 887, 7033],
 [27, 0, 860, 887, 7029],
 [35, 0, 769, 804, 6396],
 [34, 0, 840, 874, 6894],
 [25, 0, 681, 706, 5610],
 [39, 0, 796, 835, 6621],
 [28, 0, 842, 870, 6838],
 [20, 0, 455, 475, 3845],
 [44, 0, 651, 695, 5585],
 [53, 0, 712, 765, 6079],
 [59, 0, 591, 650, 5142],
 [47, 0, 741, 788, 6248],
 [29, 0, 715, 744, 5888],
 [48, 0, 512, 560, 4476],
 [53, 0, 761, 814, 6410],
 [39, 0, 714, 753, 6003]]

0

In [ ]:


#기존. 같은 measure에 여러개 생성.
for filename in filenames:
    file_data = {"performance_id": filename, "features": []}
    
    # 해당 filename에 대한 데이터를 필터링합니다.
    file_df = df[df['filename'] == filename]
    
    # 고유한 feature를 찾습니다.
    features = file_df['feature'].unique()

    for feature in features:
        feature_data = {"feature_type": int(feature), "intervals": []}
        
        # 해당 feature에 대한 데이터를 필터링합니다.
        feature_df = file_df[file_df['feature'] == feature]
        
        for _, row in feature_df.iterrows():
            interval = {
                "interval": [int(row['start']), int(row['end'])],
                "value": float(row['value'])
            }
            value_id = float(row['value'])
            

                
            
            feature_data["intervals"].append(interval)
        
        file_data["features"].append(feature_data)
    output.append(file_data)

# JSON 객체를 생성합니다.
json_data = {"output": output}

# JSON 파일로 저장합니다.
with open('output.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

# 최종 결과만 출력
print("JSON 파일이 성공적으로 생성되었습니다.")

[[0, 0, 0],
 [6125, 56588, 0],
 [4523, 42296, 0],
 [4374, 40852, 0],
 [5978, 56384, 0],
 [5917, 55620, 0],
 [4189, 38928, 0],
 [4535, 42376, 0],
 [3457, 32208, 0],
 [3532, 33288, 0],
 [4078, 38244, 0],
 [2155, 20252, 0],
 [2661, 24940, 0],
 [3326, 31152, 0],
 [2700, 25364, 0],
 [4056, 38184, 0],
 [3471, 32284, 0],
 [2099, 19732, 0],
 [3169, 29292, 0],
 [3599, 34348, 0]]